Apply FINCHES on co-expressed ADs and CoAs.

Get ADs for co-expressed TFs on MED12/12L.

In [52]:
import pandas as pd
import numpy as np

In [53]:
MED12L_TF = pd.read_csv("data/mediator/MED12L_5thpercentile_TFs.csv")
MED12_TF = pd.read_csv("data/mediator/MED12_similar_TFs.csv")

In [54]:
#Get list of ADs, with uniprot ID, Gene name, ensemble, AD sequence
ADs = pd.read_csv("~/Desktop/StallerRotation/TF_list/original_data/known_ADs_considering_isoforms_and_canonical.csv") 
ADs_mapped = pd.read_csv("~/Desktop/StallerRotation/cofactors_humanproteinatlas/data/TF-AD-RD/ADs_mapped.tsv", sep="\t")
ADs_mapped["Bgee"] = ADs_mapped["Bgee"].apply(lambda x: x.replace(";", "") if isinstance(x, str) else x)

ADs_selected = ADs[["uniprotID", "Gene", "Start", "End", "ProteinRegionSeq"]]
ADs_selected = ADs_selected.merge(ADs_mapped.set_index("Entry")[["Bgee"]], left_on="uniprotID", right_index=True, how="left")
ADs_selected = ADs_selected.rename({"ProteinRegionSeq": "Sequence", "Bgee": "Ensembl"}, axis=1)
ADs_selected.head()

,uniprotID,Gene,Start,End,Sequence,Ensembl
0,Q6UWZ7,ABRAXAS1,121,200,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,ENSG00000163322
1,Q8IUX7,AEBP1,1088,1158,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,ENSG00000106624
2,Q8WYP5,AHCTF1,1445,1698,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,ENSG00000153207
3,P35869,AHR,118,126,LLQALNGFV,ENSG00000106546
4,P35869,AHR,266,268,FAI,ENSG00000106546


In [72]:
MED12L_TF_ADs = ADs_selected.merge(MED12L_TF.set_index("Ensembl")[["Distance from MED12L"]], left_on="Ensembl", right_index=True, how="inner")
MED12_TF_ADs = ADs_selected.merge(MED12_TF.set_index("Ensembl")[["Distance from MED12"]], left_on="Ensembl", right_index=True, how="inner")

MED12L_TF_ADs["Classification"] = "AD"
MED12_TF_ADs["Classification"] = "AD"

MED12L_TF_non_ADs = ADs_selected[[x not in MED12L_TF["Ensembl"].to_list() for x in ADs_selected["Ensembl"]]]
MED12_TF_non_ADs = ADs_selected[[x not in MED12_TF["Ensembl"].to_list() for x in ADs_selected["Ensembl"]]]

MED12L_TF_non_ADs["Classification"] = "non-AD"
MED12_TF_non_ADs["Classification"] = "non-AD"

MED12_TF_ADs = pd.concat([MED12_TF_ADs, MED12_TF_non_ADs])
MED12L_TF_ADs = pd.concat([MED12L_TF_ADs, MED12L_TF_non_ADs])

MED12L_TF_ADs.to_csv("helperdata/MED12L_similar_ADs.csv", index=False)
MED12_TF_ADs.to_csv("helperdata/MED12_similar_ADs.csv", index=False)

/var/folders/rq/l7tf3_nx0ml2t2rb_tgxvm280000gn/T/ipykernel_30764/1983213300.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MED12L_TF_non_ADs["Classification"] = "non-AD"
/var/folders/rq/l7tf3_nx0ml2t2rb_tgxvm280000gn/T/ipykernel_30764/1983213300.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MED12_TF_non_ADs["Classification"] = "non-AD"


,uniprotID,Gene,Start,End,Sequence,Ensembl,Distance from MED12L,Classification
20,Q9HBZ2,ARNT2,524,717,QGSPFPSGHSGKAFSSSVVHVPGVNDIQSSSSTGQNMSQISRQLNQ...,ENSG00000172379,295.190278,AD
21,O00327,ARNTL,543,626,GGTPDIPSSGLLSGQAQENPGYPYSDSSSILGENPHIGIDMIDNDQ...,ENSG00000133794,456.726373,AD
91,Q9Y222,DMTF1,87,170,VTMTATTEVADDEVTEGTVTQIQILQNEQLDEISPLGNEEVSAVSQ...,ENSG00000135164,424.563152,AD
92,Q9Y222,DMTF1,458,760,MAALQIPVQITHVSSADSPATVDSETITLNSGTLQTFEILPSFHLQ...,ENSG00000135164,424.563152,AD
93,Q9Y222,DMTF1,1,86,MSTVEEDSDTVTVETVNSVTLTQDTEGNLILHCPQNEADEIDSEDS...,ENSG00000135164,424.563152,AD
